In [8]:
import json
import os
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
import sknetwork as skn
from collections import Counter
import scipy

In [9]:
adjacency_matrix = {}
with open('adjacency_matrix.json') as f:
    adjacency_matrix = json.load(f)
adjacency = scipy.sparse.csr_matrix(adjacency_matrix)
adjacency

In [10]:
B = bipartite.from_biadjacency_matrix(adjacency)

In [13]:
from sknetwork.clustering import Louvain

louvain = Louvain()
labels = louvain.fit_predict(adjacency)
len(set(labels))

10